In [ ]:
import numpy as np
from astropy import units as u
import csv
import configparser
import os
import subprocess

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
from matplotlib.pyplot import figure, show, subplots
from matplotlib import pyplot as plt


In [ ]:
def specfluxden_to_jansky(wavelength,specfluxden):
    """Convert spectral flux density (erg/s/cm^2/nm) to Janskys, using wavelength in microns"""
    wavelength = wavelength * u.micron
    specfluxden = specfluxden * u.erg / (u.s * u.cm**2 * u.nm)
    return specfluxden.to(u.Jy, equivalencies=u.spectral_density(wavelength)).value

def distance_correction(specfluxden,distance):
    """"Reduces intensity through inverse-square law, using observing distance (in parsec)"""
    return specfluxden*(6378/(distance*3.09E13))**2 # AGNI values are observed at 1 Earth radius

def spectrum_cutoff(bandcenter,bandflux):
    """Cut spectrum to only include 4-18.5 microns"""
    for i in range(len(bandcenter)):
        if bandcenter[i] > 4.0:
            lower_index = i-1
            break
    for j in range(len(bandcenter)):
        if bandcenter[j] > 18.5:
            upper_index = j+1
            break
    return bandcenter[lower_index:upper_index], bandflux[lower_index:upper_index]


In [ ]:
def agni_in_lifesim(agnifilename,distance,integrationtime):
    """Take one AGNI spectrum, convert it to Janskys, and run it through LIFEsim"""
    
    # get AGNI data and convert it to Janskys
    data_path = "../output/AGNI/AGNI_textfiles/" + agnifilename 
    bandcenter = np.genfromtxt(data_path,skip_header=1,delimiter=",")[:,0]/1000 # microns
    bandflux = np.genfromtxt(data_path,skip_header=1,delimiter=",")[:,1] # erg / (s * cm**2 * nm)
    bandflux_converted = specfluxden_to_jansky(bandcenter,bandflux)
    bandflux_distancecorrected = distance_correction(bandflux_converted,distance)
    bandcenter_cut, bandflux_cut = spectrum_cutoff(bandcenter,bandflux_distancecorrected)
    
    # write Jansky conversion to txt file
    file_destination = "../output/LIFEsim/clean_spectra/textfiles/" + "clean" + agnifilename[4:-4] + "_" + str(distance) + ".txt"
    with open(file_destination, 'w') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerows(zip(bandcenter_cut,bandflux_cut))

    # make images of clean spectra
    fig = figure(figsize=(8,6))
    frame1 = fig.add_subplot(1,1,1)
    frame1.plot(bandcenter_cut,bandflux_cut)
    frame1.set_xlabel('wavelength (microns)')
    frame1.set_ylabel('Janskys')
    frame1.set_yscale('log')
    frame1.set_xlim(4,18.5)
    filename = '../output/LIFEsim/clean_spectra/images/'+ "clean" + agnifilename[4:-4] + "_" + str(distance) + '.png'
    fig.savefig(filename)

    # Edit LIFEsim config
    config = configparser.ConfigParser(comment_prefixes="#",allow_no_value=True)
    config.read('GEMINI/init_simulation.cfg')
    config['simulation']['simulation_distance-to-planet'] = str(distance)
    config['simulation']['simulation_distance-to-star'] = str(distance)
    config['simulation']['simulation_integration-time'] = str(integrationtime)
    config['planet']['planet_pathtospectrum'] = file_destination
    with open('GEMINI/init_simulation.cfg', 'w') as configfile:
        config.write(configfile)
    
    # run LIFEsim through GEMINI
    prompt = "source /dataserver/users/formingworlds/borgmann/.LIFEsim_venv/bin/activate"
    subprocess.run(prompt, shell=True)
    prompt = "python GEMINI/gemini.py"
    subprocess.run(prompt, shell=True)

    # copy GEMINI output to my own output folder
    prompt = "cp GEMINI/outputdir/output-figure.pdf ../output/LIFEsim/observed_spectra/images/" + "observed" + agnifilename[4:-4] + "_" + str(distance) + ".pdf"
    subprocess.run(prompt, shell=True)
    prompt = "cp GEMINI/outputdir/data.txt ../output/LIFEsim/observed_spectra/textfiles/" + "observed" + agnifilename[4:-4] + "_" + str(distance) + ".txt"
    subprocess.run(prompt, shell=True)

In [ ]:
# convert AGNI data to be used for LIFEsim
data_path = "../output/AGNI/AGNI_textfiles"
files = [f for f in os.listdir(data_path)]
for i in range(len(files)):
    for j in [5,10,15,20]:
        agni_in_lifesim(files[i],j,36000)

In [ ]:
# find lowest value and highest Jansky values of all spectra
# purpose: to have the same y-range in every image, while keeping every full spectrum in view

data_path = "../output/LIFEsim/clean_spectra/textfiles/"
files = [f for f in os.listdir(data_path)]

all_bandfluxes = []

for i in range(len(files)):
    image_path = data_path + files[i]
    bandflux = np.genfromtxt(image_path)[:,1]
    all_bandfluxes.extend(bandflux)
    print(files[i])
    print(min(bandflux))

plotmin = min(all_bandfluxes)
plotmax = max(all_bandfluxes)
print(plotmin)
print(plotmax)

In [ ]:
# save individual images
plt.ioff() # supress notebook output

data_path = "../output/LIFEsim/clean_spectra/textfiles/"
files = [f for f in os.listdir(data_path)]

for i in range(len(files)):
    image_path = data_path + files[i]
    bandcenter = np.genfromtxt(image_path)[:,0]
    bandflux = np.genfromtxt(image_path)[:,1]

    fig = figure(figsize=(8,6))
    frame1 = fig.add_subplot(1,1,1)
    frame1.plot(bandcenter,bandflux)
    frame1.set_xlabel('wavelength (microns)')
    frame1.set_ylabel('Janskys')
    #frame1.set_xscale('log')
    frame1.set_yscale('log')
    frame1.set_xlim(4,18.5)
    #frame1.set_ylim(5E-10,3E-6)
    frame1.set_ylim(plotmin,plotmax)
    filename = '../output/LIFEsim/clean_spectra/images/' + files[i][:-4] + '.png'
    fig.savefig(filename)